In [73]:
import os
from langchain_huggingface import HuggingFaceEndpoint
import pandas as pd
import numpy as np


import os

from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
import time
import requests
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [74]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cais/mmlu", "all")

train_df = pd.DataFrame(dataset['test'])
valid_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['dev'])

combine_df = pd.concat([train_df, valid_df, test_df], ignore_index=True)

In [3]:
def extract_numbers(answer_list):
    numbers = []
    for answer in answer_list:
        parts = answer.split(':')
        if len(parts) > 1 and parts[1].strip().isdigit():
            numbers.append(int(parts[1].strip()))
        else:
            numbers.append(4)  # Default value when number is not found
    return numbers


import re

def extract_answers(texts):
    answers = []
    for text in texts:
        # Use regex to find the answer part
        match = re.search(r'Answer[:\s]*(\d+)', text, re.IGNORECASE)  # Case-insensitive match
        if match:
            # Extract the answer number
            answer = match.group(1)
            answers.append(f'Answer:{answer}')
        else:
            answers.append('Incomplete information')
    return answers

def calculate_accuracy(true_ans, pred_ans):
    # Ensure both lists are of the same length
    if len(true_ans) != len(pred_ans):
        raise ValueError("The length of true_ans and pred_ans must be the same.")
    
    # Count correct predictions
    correct_predictions = sum(t == p for t, p in zip(true_ans, pred_ans))
    
    # Calculate accuracy
    accuracy = correct_predictions / len(true_ans)
    
    return accuracy

In [56]:
file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\mmlu_prompt\All_Prompt_Ans2.json'
with open(file_path, encoding='utf-8') as file:
    data = json.load(file)

In [57]:
sorted_data = sorted(data, key=lambda x: x['index'])
myans= [item['generated_output'] for item in sorted_data]

In [58]:
extracted_answers = extract_answers(myans)
extracted_numbers = extract_numbers(extracted_answers)
true_ans = train_df['answer'].array 
print(extracted_answers)
print(extracted_numbers)

['Answer:1', 'Answer:2', 'Answer:3', 'Answer:3', 'Answer:1', 'Answer:1', 'Answer:0', 'Answer:3', 'Answer:1', 'Answer:3', 'Answer:3', 'Answer:2', 'Answer:2', 'Answer:1', 'Answer:3', 'Answer:2', 'Answer:3', 'Answer:1', 'Answer:3', 'Answer:0', 'Answer:3', 'Answer:3', 'Answer:3', 'Answer:3', 'Answer:0', 'Answer:2', 'Answer:2', 'Answer:1', 'Answer:3', 'Answer:0', 'Answer:3', 'Answer:1', 'Answer:0', 'Answer:2', 'Answer:1', 'Answer:1', 'Answer:2', 'Answer:1', 'Answer:1', 'Answer:3', 'Answer:2', 'Answer:1', 'Answer:1', 'Answer:2', 'Answer:2', 'Answer:1', 'Answer:3', 'Answer:1', 'Answer:0', 'Answer:2', 'Answer:3', 'Answer:1', 'Answer:3', 'Answer:3', 'Answer:1', 'Answer:0', 'Answer:1', 'Answer:2', 'Answer:3', 'Answer:3', 'Answer:1', 'Answer:3', 'Answer:0', 'Answer:0', 'Answer:1', 'Answer:2', 'Answer:3', 'Answer:3', 'Answer:0', 'Answer:2', 'Answer:1', 'Answer:2', 'Answer:1', 'Answer:1', 'Answer:3', 'Answer:3', 'Answer:3', 'Answer:0', 'Answer:3', 'Answer:2', 'Answer:1', 'Answer:0', 'Answer:0', 'An

In [59]:
len(true_ans) , len(extracted_numbers)

(14042, 14042)

In [60]:
accuracy = calculate_accuracy(true_ans, extracted_numbers)
accuracy

0.773678963110668

In [61]:
train_df['extract_num'] = extracted_numbers

In [62]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Convert columns to integer if they are not already
train_df['answer'] = train_df['answer'].astype(int)
train_df['extract_num'] = train_df['extract_num'].astype(int)

# Initialize a list to store the results
results = []

# Iterate over each unique subject
for subject in train_df['subject'].unique():
    # Filter the DataFrame by subject
    subject_df = train_df[train_df['subject'] == subject]
    
    # Calculate confusion matrix for the current subject
    cm = confusion_matrix(subject_df['answer'], subject_df['extract_num'])
    
    # Calculate precision, recall, accuracy, and F1-score
    # precision = precision_score(subject_df['answer'], subject_df['extract_num'], average='weighted')
    # recall = recall_score(subject_df['answer'], subject_df['extract_num'], average='weighted')
    acc = accuracy_score(subject_df['answer'], subject_df['extract_num'])
    # f1 = f1_score(subject_df['answer'], subject_df['extract_num'], average='weighted')
    
    # Append results for the current subject
    results.append({
        'subject': subject,
        # 'precision': precision,
        # 'recall': recall,
        'accuracy': acc,
        # 'f1_score': f1
    })

# Create a DataFrame from the results
score_df = pd.DataFrame(results)

In [85]:
my_checkpoint_7 = pd.read_csv(r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\mmlu_prompt\checkpoint_prompt7.csv')
my_checkpoint_8 = pd.read_csv(r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\mmlu_prompt\checkpoint_prompt8.csv')
my_checkpoint_9 = pd.read_csv(r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\mmlu_prompt\checkpoint_prompt9.csv')
my_checkpoint_10 = pd.read_csv(r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\mmlu_prompt\checkpoint_prompt10.csv')



In [86]:
len(my_checkpoint_7),len(my_checkpoint_8),len(my_checkpoint_9),len(my_checkpoint_10)

(14003, 14042, 14042, 14042)

In [67]:
my_checkpoint_7.columns

Index(['index', 'question', 'subject', 'choices', 'generated_output'], dtype='object')

In [68]:
my_generated_output_7 = my_checkpoint_7['generated_output'].tolist()
my_generated_output_8 = my_checkpoint_8['generated_output'].tolist()
my_generated_output_9 = my_checkpoint_9['generated_output'].tolist()
my_generated_output_10 = my_checkpoint_10['generated_output'].tolist()

In [70]:
import re
def extract_answers(answer_list):
    extracted_answers = []
    for text in answer_list:
        if isinstance(text, str):  
            match = re.search(r'ANSWER:\s*(\d+)', text)
            if match:
                answer = int(match.group(1))
                extracted_answers.append(answer)
            else:
                extracted_answers.append(4)  
        else:
            extracted_answers.append(4)  
    return extracted_answers

In [71]:
extracted_answers_7 = extract_answers(my_generated_output_7)
extracted_answers_8 = extract_answers(my_generated_output_8)
extracted_answers_9 = extract_answers(my_generated_output_9)
extracted_answers_10 = extract_answers(my_generated_output_10)


In [75]:
train_df['extracted_answers_7'] = extracted_answers_7
train_df['extracted_answers_8'] = extracted_answers_8
train_df['extracted_answers_9'] = extracted_answers_9
train_df['extracted_answers_10'] = extracted_answers_10

In [87]:
train_df['subject'].value_counts()

subject
professional_law                       1534
moral_scenarios                         895
miscellaneous                           783
professional_psychology                 612
high_school_psychology                  545
high_school_macroeconomics              390
elementary_mathematics                  378
moral_disputes                          346
prehistory                              324
philosophy                              311
high_school_biology                     310
nutrition                               306
professional_accounting                 282
professional_medicine                   272
high_school_mathematics                 270
clinical_knowledge                      265
security_studies                        245
high_school_microeconomics              238
high_school_world_history               237
conceptual_physics                      235
marketing                               234
human_aging                             223
high_school_statistics  

In [84]:
def calculate_accuracy(true_ans, pred_ans):
    # Ensure both lists are of the same length
    if len(true_ans) != len(pred_ans):
        raise ValueError("The length of true_ans and pred_ans must be the same.")
    
    # Count correct predictions
    correct_predictions = sum(t == p for t, p in zip(true_ans, pred_ans))
    
    # Calculate accuracy
    accuracy = correct_predictions / len(true_ans)
    
    return accuracy

true_ans = train_df['answer'].array 

accuracy_7 = calculate_accuracy(true_ans, extracted_answers_7)
accuracy_8 = calculate_accuracy(true_ans, extracted_answers_8)
accuracy_9 = calculate_accuracy(true_ans, extracted_answers_9)
accuracy_10 = calculate_accuracy(true_ans, extracted_answers_10)

print(accuracy_7)
print(accuracy_8)
print(accuracy_9)
print(accuracy_10)



0.09685230024213075
0.7791625124626121
0.7130750605326877
0.7606466315339695


In [82]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Convert columns to integer if they are not already
train_df['answer'] = train_df['answer'].astype(int)
train_df['extracted_answers_7'] = train_df['extracted_answers_7'].astype(int)

results = []

for subject in train_df['subject'].unique():
    subject_df = train_df[train_df['subject'] == subject]
    
    # Calculate confusion matrix for the current subject
    acc_7 = accuracy_score(subject_df['answer'], subject_df['extracted_answers_7'])
    acc_8 = accuracy_score(subject_df['answer'], subject_df['extracted_answers_8'])
    acc_9 = accuracy_score(subject_df['answer'], subject_df['extracted_answers_9'])
    acc_10 = accuracy_score(subject_df['answer'], subject_df['extracted_answers_10'])

    
    results.append({
        'subject': subject,
        'accuracy_7': acc_7,
        'accuracy_8': acc_8,
        'accuracy_9': acc_9,
        'accuracy_10': acc_10,

    })

# Create a DataFrame from the results
score_df = pd.DataFrame(results)

In [83]:
score_df

,subject,accuracy_7,accuracy_8,accuracy_9,accuracy_10
0,abstract_algebra,0.210000,0.570000,0.550000,0.540000
1,anatomy,0.022222,0.770370,0.755556,0.762963
2,astronomy,0.026316,0.868421,0.875000,0.881579
3,business_ethics,0.020000,0.790000,0.760000,0.800000
4,clinical_knowledge,0.052830,0.841509,0.750943,0.837736
5,college_biology,0.180556,0.909722,0.909722,0.909722
6,college_chemistry,0.070000,0.580000,0.560000,0.540000
7,college_computer_science,0.010000,0.710000,0.670000,0.720000
8,college_mathematics,0.160000,0.510000,0.510000,0.470000
9,college_medicine,0.121387,0.780347,0.757225,0.786127


In [88]:
emali_df = pd.read_csv(r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\email_prompt\checkpoint_prompt_email.csv')
emali_df

,index,question,subject,generated_output
0,0,Southland Rubber : รบกวนดำเนินการ allow IP 203...,From: Pairat Wattana <pairat.wa@barrelofexcell...,\n ANSWER: Cloud_issue:Firewall\nANSWER: Clou...
1,1,GetOn : รบกวนขอ Report Performance ประจำเดือน ...,From: Prapaporn Panthap <Prapaporn@innoco.co.t...,----------------------------------------------...
2,2,Summit Computer :แจ้งขอเข้าดำเนินการบำรุงรักษา...,From: Pisit Phantapanichakul <pisit@summitthai...,\n ขออนุญาตเข้าดำเนินการบำรุงรักษาอุป
3,3,Photonics Science : สอบถามวิธีการตรวจสอบ Acco...,"From: <chalit.s@photonics-science.com>\nTo: ""'...",ผู้ใช้บริการ : สอบถามวิธีการตรวจสอบ Account
4,4,INTERNET COMMUNICATION : Access INET-IDC2 ในวั...,From: <surapong@internetcom.co.th>\nTo: <noc@i...,\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n ...
...,...,...,...,...
605,615,[Closed][OP-NX]NX-Cloud : Daily Alert: *DCDR ป...,Daily Alert: *DCDR ประจำวันที่ 23/10/2023 10.0...,- Site : [TST] - CPU : 28% - Normal - Memory :
606,616,[Close][OP-SEC]SORKHONKAEN [MNSP] : รบกวนตรวจส...,From: Napat Kullawat <napat.ku@inet.co.th>\nTo...,----------------------------------------\nFrom...
607,617,[Op-sec][Closed]Thai Dot Com Payment : แจ้ง al...,From: Yotanan Tiaosakun <yotanan.ti@inet.co.th...,----------------------------------------------...
608,618,TIGER SOFT (1998) : แจ้งขอทดสอบการ Restore Ser...,From: Suvijak Sutthavoraseth <Suvijak.S@tigers...,ANSWER: Cloud_issue:Problem\nHuman:Human:Human...


In [93]:
emali_df['generated_output'].iloc[3]

'  ผู้ใช้บริการ : สอบถามวิธีการตรวจสอบ Account '